In [ ]:
CREATE DATABASE A4;

In [ ]:
use A4;

In [ ]:
CREATE TABLE GroupTable(
    gid INT PRIMARY KEY,
    gname VARCHAR(255) NOT NULL
);


In [ ]:
CREATE TABLE StudentTable(
    sid INT PRIMARY KEY,
    sname VARCHAR(30),
    gid INT FOREIGN KEY REFERENCES GroupTable(gid)
);

In [ ]:
CREATE TABLE FavSubjectTable(
    sid INT FOREIGN KEY REFERENCES StudentTable(sid),
    gid INT FOREIGN KEY REFERENCES GroupTable(gid),
    subjectName VARCHAR(50),
    PRIMARY KEY(sid, gid)
)

In [ ]:
CREATE PROCEDURE InsertGroups -- insert 500 groups
AS
BEGIN
    SET NOCOUNT ON; -- mute messages
    DECLARE @i INT = 1;
    -- Loop to insert 200 groups
    WHILE @i <= 500 
    BEGIN
        -- Insert statement
        INSERT INTO GroupTable (gid, gname) VALUES (
            @i,
            CONCAT('Group_', @i) -- Generating a unique name "Group_1", "Group_2", etc.
        ) 
        -- Increment the student ID counter
        SET @i = @i + 1;
    END;
END;

In [ ]:
CREATE PROCEDURE InsertStudents -- insert 10.000 students
AS 
BEGIN
    SET NOCOUNT ON; -- mute messages
    DECLARE @i INT = 1;
    DECLARE @groupNum INT = 1;
    -- Loop to insert 10,000 students
    WHILE @i <= 10000 
    BEGIN
        -- Insert statement
        INSERT INTO StudentTable (sid, sname, gid) VALUES (
            @i,
            CONCAT('Student_', @i),  -- Generating a unique name "Student_1", "Student_2", etc.
            @groupNum
        );
        
        -- Increment the student ID counter
        SET @i = @i + 1;

        -- Cycle through group IDs from 1 to 500
        SET @groupNum = @groupNum + 1;
        IF @groupNum > 500 
        BEGIN
            SET @groupNum = 1;
        END;
    END;
END;

In [ ]:
CREATE PROCEDURE InsertFavSubjects -- insert 2.000 fav subjects for the first 2.000 students
AS
BEGIN
    SET NOCOUNT ON; -- mute messages
    DECLARE @i INT = 1;
    DECLARE @groupNum INT = 1;
    -- Loop to insert 10,000 students
    WHILE @i <= 2000
    BEGIN
        DECLARE @subjectName VARCHAR(50);
        IF @i % 4 = 0 
        BEGIN
            SET @subjectName = 'Math';
        END
        ELSE IF @i % 4 = 1 
        BEGIN
            SET @subjectName = 'English';
        END
        ELSE IF @i % 4 = 2
        BEGIN
            SET @subjectName = 'History';
        END
        ELSE IF @i % 4 = 3
        BEGIN
            SET @subjectName = 'Science';
        END;
        
        -- Insert statement
        INSERT INTO FavSubjectTable (sid, gid, subjectName) VALUES (
            @i,
            @groupNum,
            @subjectName  -- cycle through Math, English, History, Science
        );
        
        -- Increment the student ID counter
        SET @i = @i + 1;

        -- Cycle through group IDs from 1 to 500
        SET @groupNum = @groupNum + 1;
        IF @groupNum > 500
        BEGIN
            SET @groupNum = 1;
        END;
    END;
END;

In [ ]:
CREATE PROCEDURE InsertAll -- we will not use it, just made it for better convenience
AS
BEGIN
    SET NOCOUNT ON; -- mute messages
    EXEC InsertGroups;
    EXEC InsertStudents;
    EXEC InsertFavSubjects;
END;

In [ ]:
CREATE PROCEDURE ResetTables
AS
BEGIN
    SET NOCOUNT ON; -- mute messages
    DELETE FROM FavSubjectTable;    
    DELETE FROM StudentTable;
    DELETE FROM GroupTable;
    
END;

In [ ]:
CREATE VIEW ScienceAsFavSubject AS -- view with a SELECT statement operating on one table
    SELECT sid, gid FROM FavSubjectTable WHERE subjectName = 'Science';

In [ ]:
CREATE VIEW StudentsFromGroup1 AS -- view with a SELECT statement that operates on at least 2 different tables and contains at least one JOIN operator
    SELECT 
        StudentTable.sid,
        StudentTable.sname,
        GroupTable.gname
    FROM 
        StudentTable
    INNER JOIN 
        GroupTable ON StudentTable.gid = GroupTable.gid AND GroupTable.gid = 1;


In [ ]:
CREATE VIEW FavSubjectCount AS -- a view with SELECT statement that has a GROUP BY clause, operates on at least 2 different tables and uses at least one JOIN operator
SELECT 
    FavSubjectTable.subjectName,
    COUNT(*) AS StudentCount
FROM 
    FavSubjectTable
INNER JOIN 
    StudentTable ON StudentTable.sid = FavSubjectTable.sid
GROUP BY 
    FavSubjectTable.subjectName;



In [ ]:
CREATE TABLE Tables(
    tid INT PRIMARY KEY,
    tname VARCHAR(255) NOT NULL
);

In [ ]:
INSERT INTO Tables (tid, tname) VALUES 
    (1, 'GroupTable'),
    (2, 'StudentTable'),
    (3, 'FavSubjectTable');

In [ ]:
CREATE TABLE Tests (
    tid INT PRIMARY KEY,
    executionTime DATETIME,
    testName VARCHAR(20),
    testDesc VARCHAR(255),
)